# 🏐 VOLLEY-REF AI - Exploration Notebook

Ce notebook permet d'explorer et de tester les différents composants du système VOLLEY-REF AI.

## 1. Setup & Imports

In [ ]:
import sys
sys.path.insert(0, '..')

import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# VOLLEY-REF AI modules
from src.config import CourtConfig, ModelConfig, DecisionConfig
from src.court_detector import CourtDetector
from src.ball_tracker import BallTracker
from src.landing_predictor import LandingPredictor
from src.decision_engine import DecisionEngine, Decision
from src.visualizer import Visualizer
from src.pipeline import VolleyRefAI

print("✓ All modules imported successfully!")

## 2. Configuration

In [ ]:
# Court configuration
court_config = CourtConfig()

print("Court Dimensions (FIVB):")
print(f"  Length: {court_config.LENGTH} cm ({court_config.LENGTH/100}m)")
print(f"  Width: {court_config.WIDTH} cm ({court_config.WIDTH/100}m)")
print(f"  Attack line: {court_config.ATTACK_LINE} cm from net")
print(f"  Line width: {court_config.LINE_WIDTH} cm")
print(f"\nKeypoints ({len(court_config.KEYPOINT_NAMES)}):")
for i, name in enumerate(court_config.KEYPOINT_NAMES):
    coords = court_config.real_keypoints[i]
    print(f"  {i}: {name} -> ({coords[0]:.0f}, {coords[1]:.0f}) cm")

## 3. Visualize Court Layout

In [ ]:
def plot_court(config: CourtConfig, figsize=(12, 6)):
    """Plot the volleyball court layout with keypoints."""
    fig, ax = plt.subplots(figsize=figsize)
    
    # Court outline
    court = plt.Rectangle((0, 0), config.LENGTH, config.WIDTH, 
                          fill=True, facecolor='green', edgecolor='white', linewidth=3)
    ax.add_patch(court)
    
    # Center line (net)
    ax.axvline(x=config.LENGTH/2, color='white', linewidth=3)
    
    # Attack lines
    ax.axvline(x=config.ATTACK_LINE, color='white', linewidth=2, linestyle='--')
    ax.axvline(x=config.LENGTH - config.ATTACK_LINE, color='white', linewidth=2, linestyle='--')
    
    # Keypoints
    keypoints = config.real_keypoints
    ax.scatter(keypoints[:, 0], keypoints[:, 1], c='red', s=100, zorder=5)
    
    for i, name in enumerate(config.KEYPOINT_NAMES):
        ax.annotate(f"{i}", (keypoints[i, 0], keypoints[i, 1]), 
                   textcoords="offset points", xytext=(5, 5), fontsize=8)
    
    ax.set_xlim(-50, config.LENGTH + 50)
    ax.set_ylim(-50, config.WIDTH + 50)
    ax.set_aspect('equal')
    ax.set_xlabel('X (cm)')
    ax.set_ylabel('Y (cm)')
    ax.set_title('Volleyball Court Layout with Keypoints')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

plot_court(court_config)

## 4. Test Decision Engine

In [ ]:
# Initialize decision engine
decision_config = DecisionConfig()
decision_engine = DecisionEngine(court_config, decision_config)

# Test various landing points
test_points = [
    (900, 450),    # Center of court (IN)
    (100, 100),    # Near corner (IN)
    (-10, 450),    # Outside left (OUT)
    (1810, 450),   # Outside right (OUT)
    (2, 450),      # Very close to line (TOO_CLOSE)
    (1798, 450),   # Very close to line (TOO_CLOSE)
]

print("Testing Decision Engine:")
print("="*60)

for point in test_points:
    decision = decision_engine.decide(point, pred_confidence=0.9)
    print(f"Point {point}: {decision}")

## 5. Visualize Test Points on Court

In [ ]:
def plot_decisions(config: CourtConfig, engine: DecisionEngine, points: list, figsize=(12, 6)):
    """Plot landing points with their decisions."""
    fig, ax = plt.subplots(figsize=figsize)
    
    # Court
    court = plt.Rectangle((0, 0), config.LENGTH, config.WIDTH, 
                          fill=True, facecolor='green', edgecolor='white', linewidth=3)
    ax.add_patch(court)
    ax.axvline(x=config.LENGTH/2, color='white', linewidth=3)
    
    # Plot points
    colors = {'IN': 'lime', 'OUT': 'red', 'TOO_CLOSE': 'yellow'}
    
    for point in points:
        decision = engine.decide(point, pred_confidence=0.9)
        color = colors[decision.verdict]
        ax.scatter(point[0], point[1], c=color, s=200, edgecolors='black', linewidth=2, zorder=5)
        ax.annotate(f"{decision.verdict}\n{decision.distance_to_line:.1f}cm", 
                   point, textcoords="offset points", xytext=(10, 10), fontsize=8)
    
    ax.set_xlim(-100, config.LENGTH + 100)
    ax.set_ylim(-100, config.WIDTH + 100)
    ax.set_aspect('equal')
    ax.set_title('Landing Points with Decisions')
    
    # Legend
    for verdict, color in colors.items():
        ax.scatter([], [], c=color, s=100, label=verdict, edgecolors='black')
    ax.legend(loc='upper right')
    
    plt.tight_layout()
    plt.show()

plot_decisions(court_config, decision_engine, test_points)

## 6. Test Visualizer

In [ ]:
# Create visualizer
visualizer = Visualizer(court_config)

# Generate 2D court view with a landing point
landing = (900, 450)  # Center of court
decision = decision_engine.decide(landing, pred_confidence=0.95)

court_2d = visualizer.draw_2d_court(
    width=600,
    height=300,
    landing=landing,
    decision=decision
)

# Convert BGR to RGB for matplotlib
court_2d_rgb = cv2.cvtColor(court_2d, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(10, 5))
plt.imshow(court_2d_rgb)
plt.title(f'2D Court View - {decision.verdict}')
plt.axis('off')
plt.show()

## 7. Simulate Trajectory

In [ ]:
def simulate_ball_trajectory(start_pos, velocity, n_frames=30, dt=1/30, gravity=500):
    """Simulate a ball trajectory with gravity."""
    trajectory = []
    x, y = start_pos
    vx, vy = velocity
    
    for _ in range(n_frames):
        trajectory.append((x, y))
        x += vx * dt
        y += vy * dt
        vy += gravity * dt  # Gravity (positive = down in image coords)
    
    return trajectory

# Simulate a trajectory
trajectory = simulate_ball_trajectory(
    start_pos=(200, 100),
    velocity=(300, -200),
    n_frames=40
)

# Plot
points = np.array(trajectory)
plt.figure(figsize=(10, 6))
plt.plot(points[:, 0], points[:, 1], 'o-', color='orange', markersize=4)
plt.scatter(points[0, 0], points[0, 1], c='green', s=100, label='Start', zorder=5)
plt.scatter(points[-1, 0], points[-1, 1], c='red', s=100, label='End', zorder=5)
plt.gca().invert_yaxis()  # Image coordinates
plt.xlabel('X (pixels)')
plt.ylabel('Y (pixels)')
plt.title('Simulated Ball Trajectory')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 8. Test Full Pipeline (if models available)

In [ ]:
# Check if models exist
court_model_path = Path('../weights/yolo_court_keypoints.pt')
ball_model_path = Path('../weights/yolo_volleyball_ball.pt')

print("Model Status:")
print(f"  Court model: {'✓ Found' if court_model_path.exists() else '✗ Not found'}")
print(f"  Ball model: {'✓ Found' if ball_model_path.exists() else '✗ Not found'}")

if court_model_path.exists() and ball_model_path.exists():
    print("\nYou can test the full pipeline!")
else:
    print("\nTo test the full pipeline, first train the models:")
    print("  1. python scripts/download_datasets.py")
    print("  2. python scripts/finetune_court.py")
    print("  3. python scripts/finetune_ball.py")

In [ ]:
# Only run if models exist
if court_model_path.exists() and ball_model_path.exists():
    # Initialize pipeline
    pipeline = VolleyRefAI(
        court_model=str(court_model_path),
        ball_model=str(ball_model_path)
    )
    print("✓ Pipeline initialized successfully!")
    
    # Create a test frame (blank)
    test_frame = np.zeros((720, 1280, 3), dtype=np.uint8)
    test_frame[:] = (50, 50, 50)  # Dark gray
    
    # Process frame
    output, decision = pipeline.process_frame(test_frame)
    
    # Display
    output_rgb = cv2.cvtColor(output, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(12, 7))
    plt.imshow(output_rgb)
    plt.title('Pipeline Output (Test Frame)')
    plt.axis('off')
    plt.show()
else:
    print("Skipping pipeline test - models not available")

## 9. Summary

This notebook demonstrated:

1. **Configuration** - Court dimensions and keypoint locations
2. **Decision Engine** - IN/OUT verdict calculation
3. **Visualizer** - 2D court rendering
4. **Trajectory Simulation** - Ball physics modeling
5. **Full Pipeline** - End-to-end processing (if models available)

Next steps:
- Download datasets: `python scripts/download_datasets.py`
- Train models: `python scripts/finetune_court.py` and `python scripts/finetune_ball.py`
- Run demo: `python scripts/demo.py --video your_video.mp4`